In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats.mstats import winsorize 

In [5]:
# read data
dataset_raw = pd.read_csv('dataset_raw.csv')

raw = pd.read_csv('./return_variables_raw_s1.csv')
eps = pd.read_csv('./return_variables_eps_s1.csv')
ret = pd.read_csv('./return_variables_ret_s.csv')
mom = pd.read_csv('./return_variables_mom_s1.csv')
betas = pd.read_csv('./betas_s.csv')
volat = pd.read_csv('./return_variables_volat_s1.csv')
total_return = pd.read_csv('total_returns.csv')

emission_variables = pd.read_csv('./emission_variables.csv')

In [6]:
emission_variables = emission_variables.drop('Unnamed: 0', axis=1)
ret = ret.drop('Unnamed: 0', axis=1)
mom = mom.drop('Unnamed: 0', axis=1)
betas = betas.drop('Unnamed: 0', axis=1)
volat = volat.drop('Unnamed: 0', axis=1)
raw = raw.drop('Unnamed: 0', axis=1)
epsgr = eps.drop('Unnamed: 0', axis=1)
total_return = total_return.drop('Unnamed: 0', axis=1)

In [7]:
ret1 = total_return[['Total Return', 'gvkey', 'fyear']]
ret1['RET1'] = ret1['Total Return']
ret1 = ret1.drop('Total Return', axis=1)

C:\Users\18906\AppData\Local\Temp\ipykernel_31840\2880006707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret1['RET1'] = ret1['Total Return']


In [8]:
month_return_variables = pd.merge(ret, mom, on=['gvkey', 'fyear', 'month'], how='left')
month_return_variables = pd.merge(month_return_variables, betas, on=['gvkey', 'fyear', 'month'], how='left')
month_return_variables = pd.merge(month_return_variables, volat, on=['gvkey', 'fyear', 'month'], how='left')

In [9]:
month_return_variables['gvkey'] = month_return_variables['gvkey'].astype(int)
month_data = month_return_variables.dropna()
month_data = month_data.groupby(['gvkey', 'fyear', 'month']).mean().reset_index()

In [10]:
month_data.describe()

,gvkey,fyear,month,RET,MOM,BETA,VOLAT
count,311477.000000,311477.000000,311477.000000,311477.000000,311477.000000,311477.000000,311477.000000
mean,61282.014948,2016.966171,6.533590,0.908934,0.011858,1.113218,0.106599
std,66250.695704,3.413579,3.451108,12.456995,0.036238,0.714470,0.060116
min,1004.000000,2011.000000,1.000000,-99.989583,-0.109408,-1.044560,0.025872
25%,11343.000000,2014.000000,4.000000,-5.191972,-0.006148,0.672637,0.064063
50%,26545.000000,2017.000000,7.000000,0.663658,0.010312,1.047581,0.091141
75%,120093.000000,2020.000000,10.000000,6.483600,0.027922,1.473452,0.132432
max,328795.000000,2022.000000,12.000000,100.000000,0.175570,4.147888,0.356740


In [11]:
# merge yearly variables
year_return_variables = pd.merge(raw, epsgr, on=['gvkey', 'fyear'], how='left')
year_return_variables = pd.merge(year_return_variables, ret1, on=['gvkey', 'fyear'], how='left')

In [12]:
# merge emission variables to yearly variables
year_data = pd.merge(emission_variables, year_return_variables, on=['gvkey', 'fyear'], how='left')
year_data = year_data.dropna()
year_data = year_data.groupby(['gvkey', 'fyear']).mean().reset_index()

In [13]:
year_data.to_csv('merged_year_data.csv', index=True)
year_data.describe()

,gvkey,fyear,total_emission_scope1,total_emission_scope2,emission_growth_scope1,emission_growth_scope2,carbon_intensity_scope1,carbon_intensity_scope2,LOGSIZE,B/M,LEVERAGE,INVEST/A,ROE,LOGPPE,SALESGR,EPSGR,RET1
count,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000,6923.000000
mean,61114.428860,2017.412538,10.398440,10.613224,0.053271,0.066361,1.234912,0.315573,8.219125,0.577480,0.275806,0.035277,7.011773,6.822731,0.012004,0.005031,17.564246
std,68833.661469,2.154190,2.693183,2.072613,0.296678,0.291442,3.674558,0.383633,1.765333,0.529926,0.215507,0.033130,33.246637,2.096803,0.329318,0.229265,54.793028
min,1004.000000,2011.000000,0.693147,2.544276,-0.587399,-0.510633,0.003999,0.008185,-4.612195,-0.040954,0.000000,0.000000,-109.224422,-2.465104,-2.260408,-1.598919,-95.067633
25%,10911.500000,2016.000000,8.623231,9.242779,-0.074218,-0.075366,0.049394,0.085381,7.053573,0.230575,0.088529,0.012366,0.510155,5.473247,-0.016017,-0.022411,-9.971983
50%,24856.000000,2018.000000,10.204522,10.610770,0.025789,0.025275,0.140882,0.176110,8.215181,0.445294,0.251059,0.025648,10.177004,6.858055,0.021591,0.003377,11.585747
75%,133768.000000,2019.000000,11.918332,11.999193,0.137210,0.155007,0.325886,0.378714,9.387678,0.759547,0.412257,0.047526,19.064401,8.150537,0.074559,0.027413,35.018690
max,328795.000000,2020.000000,18.713283,16.571679,1.124937,1.138111,19.735829,1.780249,14.627616,2.663034,0.814584,0.163615,91.432361,13.144907,1.647431,1.808117,1042.570281


In [14]:
# merge all data as monthly data
regression_data = pd.merge(month_data, year_data, on=['gvkey', 'fyear'], how='left')
regression_data = regression_data.dropna()
regression_data = regression_data.groupby(['gvkey', 'fyear', 'month']).mean().reset_index()

In [15]:
regression_data['fyear'] = regression_data['fyear'].astype(str)
regression_data['month'] = regression_data['month'].apply(lambda x: f'{x:02d}')

regression_data['datatime'] = regression_data['fyear'] + '-' + regression_data['month']

In [16]:
# GICS
gics_data = dataset_raw[['gvkey', 'GICS_level_1']]

# match by the gvkey number
gics_data['gvkey'] = gics_data['gvkey'].astype(int)
regression_data['gvkey'] = regression_data['gvkey'].astype(int)

# merge gics_data to regression_data
regression_data = regression_data.merge(gics_data, on='gvkey', how='left')

C:\Users\18906\AppData\Local\Temp\ipykernel_31840\5735994.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gics_data['gvkey'] = gics_data['gvkey'].astype(int)


In [17]:
regression_data = regression_data.drop_duplicates(keep='last')
regression_data.to_csv('merged_data_sss.csv', index=True)
regression_data.describe()

,gvkey,RET,MOM,BETA,VOLAT,total_emission_scope1,total_emission_scope2,emission_growth_scope1,emission_growth_scope2,carbon_intensity_scope1,carbon_intensity_scope2,LOGSIZE,B/M,LEVERAGE,INVEST/A,ROE,LOGPPE,SALESGR,EPSGR,RET1
count,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000,81605.000000
mean,61120.654923,1.285111,0.010517,1.195033,0.103849,10.411239,10.627489,0.053134,0.065692,1.234506,0.314301,8.235747,0.577298,0.275880,0.035363,7.217507,6.841127,0.011243,0.004939,17.425595
std,69006.711553,12.473213,0.032102,0.727854,0.057046,2.689473,2.067171,0.296586,0.291304,3.678397,0.381090,1.762027,0.530395,0.215301,0.033117,33.035337,2.093872,0.328364,0.227102,53.736741
min,1004.000000,-87.697842,-0.109408,-1.044560,0.025872,0.693147,2.544276,-0.587399,-0.510633,0.003999,0.008185,-4.612195,-0.040954,0.000000,0.000000,-109.224422,-2.465104,-2.260408,-1.598919,-95.067633
25%,10860.000000,-4.819277,-0.005882,0.757661,0.063082,8.641213,9.258057,-0.074867,-0.076141,0.050406,0.085381,7.070696,0.230611,0.088988,0.012404,0.751054,5.498769,-0.015899,-0.022173,-9.791332
50%,24783.000000,1.053142,0.010596,1.119520,0.090176,10.219274,10.620766,0.025733,0.025094,0.141114,0.176111,8.231717,0.444195,0.251456,0.025776,10.303030,6.880971,0.021430,0.003362,11.590107
75%,133869.000000,6.870229,0.026370,1.546813,0.129241,11.925332,12.018952,0.137206,0.154444,0.324883,0.377914,9.407562,0.758144,0.412147,0.047656,19.134897,8.165563,0.073998,0.027182,34.970843
max,328795.000000,99.191686,0.175570,4.147888,0.356740,18.713283,16.571679,1.124937,1.138111,19.735829,1.780249,14.627616,2.663034,0.814584,0.163615,91.432361,13.144907,1.647431,1.808117,1042.570281
